# DS4PH Term 4 Capstone Project
Group Members: Tenzin Lhaksampa and Archana Balan  
Code to preprocess data for the app dashboard

In [13]:
# Import packages
import streamlit as st
import pandas as pd
import numpy as np
import folium
import plotly as py 
from importlib import import_module

In [15]:
# read in data
dat = pd.read_excel("./data/maltreatment_data.xlsx")
dat.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/maltreatment_data.xlsx'

In [3]:
# check columns 
print(dat.columns)
print("\n", dat.shape)

Index(['LocationType', 'Location', 'LocationCode', 'Category', 'TimeFrame',
       'DataFormat', 'Data'],
      dtype='object')

 (5088, 7)


In [4]:
# only retain numbers (not percentages)
dat = dat[dat['DataFormat'] == 'Number']

# convert cases reported to a numeric value
dat['Data'] = pd.to_numeric(dat['Data'], errors='coerce')

In [5]:
# rename and retain only relevant columns 
dat.rename({'LocationCode':'State', 
            'Category':'Type', 
            'TimeFrame':'Year', 
            'Data':'Case'}, 
           axis=1, 
           inplace=True)

# drop NA values
dat = dat.dropna()

# print data frame shape and column names
print(dat.columns)
print("\n", dat.shape)

Index(['LocationType', 'Location', 'State', 'Type', 'Year', 'DataFormat',
       'Case'],
      dtype='object')

 (2185, 7)


In [6]:
# drop missing data 
dat = dat[dat['Type'] != 'Other/missing maltreatment type']
dat.shape

(1982, 7)

In [7]:
# drop US data
dat = dat[dat['State'] != 'US']
dat.shape

(1942, 7)

In [8]:
# Capitalize words in the 'Type' column using str.title() method
dat['Type'] = dat['Type'].str.title()
dat.head() # check

,LocationType,Location,State,Type,Year,DataFormat,Case
0,State,Alabama,AL,Emotional Abuse,2021,Number,19.0
1,State,Alabama,AL,Medical Neglect,2021,Number,79.0
2,State,Alabama,AL,Neglect,2021,Number,5011.0
4,State,Alabama,AL,Physical Abuse,2021,Number,6062.0
5,State,Alabama,AL,Sexual Abuse,2021,Number,2103.0


In [9]:
# merge "Neglect" and "Medical Neglect" into only "Neglect" 
dat['Type'] = dat['Type'].replace('Medical neglect', 'Neglect')
dat = dat.groupby(['State', 'Type', 'Year']).agg({'Case': 'sum'}).reset_index()

print(dat.shape) # check shape
print("\n", dat.head()) # check dataset

(1881, 4)

   State             Type  Year   Case
0    AK  Emotional Abuse  2015  718.0
1    AK  Emotional Abuse  2016  985.0
2    AK  Emotional Abuse  2017  770.0
3    AK  Emotional Abuse  2018  684.0
4    AK  Emotional Abuse  2019  994.0


In [10]:
# confirm data merge using MD as an example
md_dat = dat[(dat['State'] == 'MD') & (dat['Type'] == 'Neglect')]
md_dat.head(10)

,State,Type,Year,Case
703,MD,Neglect,2015,4020.0
704,MD,Neglect,2016,4155.0
705,MD,Neglect,2017,4568.0
706,MD,Neglect,2018,4779.0
707,MD,Neglect,2019,4648.0
708,MD,Neglect,2020,4220.0
709,MD,Neglect,2021,3491.0
710,MD,Neglect,2022,3691.0


## Total Maltreatment

In [11]:
# Calculate the total case count of child maltreatment across types
# Group by "State" and "Type", and sum the "Case" column
sum_dat = dat.groupby(['State', 'Year'])['Case'].sum().reset_index()

# generate the max number of cases 
max_value = dat['Case'].max()

# Display the resulting DataFrame
print(sum_dat.head())

print("\nHighest Total Maltreatment Case Count in the US:", max_value)

print("\n", sum_dat.shape)

  State  Year    Case
0    AK  2015  3646.0
1    AK  2016  3997.0
2    AK  2017  3449.0
3    AK  2018  3392.0
4    AK  2019  4136.0

Highest Total Maltreatment Case Count in the US: 67427.0

 (398, 3)


## Maltreatment Types

In [12]:
# data frames per maltreatment type 
emotional = dat[dat['Type'] == 'Emotional Abuse'][['State', 'Year', 'Case']]
physical = dat[dat['Type'] == 'Physical Abuse'][['State', 'Year', 'Case']]
sexual = dat[dat['Type'] == 'Sexual Abuse'][['State', 'Year', 'Case']]
neglect = dat[dat['Type'] == 'Neglect'][['State', 'Year', 'Case']]

print("Emotional Abuse\n", emotional.head(3))
print("\nPhysical Abuse\n", physical.head(3)) 
print("\nSexual Abuse\n", sexual.head(3))
print("\nNeglect\n", neglect.head(3))

Emotional Abuse
   State  Year   Case
0    AK  2015  718.0
1    AK  2016  985.0
2    AK  2017  770.0

Physical Abuse
    State  Year   Case
24    AK  2015  331.0
25    AK  2016  376.0
26    AK  2017  393.0

Sexual Abuse
    State  Year   Case
32    AK  2015  155.0
33    AK  2016  181.0
34    AK  2017  148.0

Neglect
    State  Year    Case
16    AK  2015  2358.0
17    AK  2016  2382.0
18    AK  2017  2133.0


# Save data as input for streamlit app

In [13]:
# save data to be displayed in the app's dashboard
dat.to_csv('./data/dashboard_processed.csv')
sum_dat.to_csv('./data/dashboard_data.csv')